In [1]:
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, TimeDistributed, BatchNormalization, Reshape 
from keras.layers.convolutional import Conv1D
from keras.layers.pooling import MaxPooling1D
from sklearn.metrics import r2_score, accuracy_score
import matplotlib.pyplot as plt
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
from keras import regularizers
from keras import optimizers
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler

Using TensorFlow backend.


In [2]:
y_train = pd.read_csv('../model_input/train_y.csv', index_col=0).values
y_valid = pd.read_csv('../model_input/valid_y.csv', index_col=0).values
y_test = pd.read_csv('../model_input/test_y.csv', index_col=0).values

In [3]:
df_y_train = pd.DataFrame(y_train, columns=['valence', 'arousal'])
df_y_train['label'] = 0
df_y_train.loc[(df_y_train['valence'] < df_y_train['arousal']), 'label'] = 1
y_train = df_y_train['label'].values

df_y_test = pd.DataFrame(y_test, columns=['valence', 'arousal'])
df_y_test['label'] = 0
df_y_test.loc[(df_y_test['valence'] < df_y_test['arousal']), 'label'] = 1
y_test = df_y_test['label'].values

df_y_valid = pd.DataFrame(y_valid, columns=['valence', 'arousal'])
df_y_valid['label'] = 0
df_y_valid.loc[(df_y_valid['valence'] < df_y_valid['arousal']), 'label'] = 1
y_valid = df_y_valid['label'].values

In [4]:
"""
X_train = np.load('../model_input/train_pitch_x.npz')['arr_0'].reshape(-1, 300, 12, 1)
X_test = np.load('../model_input/test_pitch_x.npz')['arr_0'].reshape(-1, 300, 12, 1)
X_valid = np.load('../model_input/valid_pitch_x.npz')['arr_0'].reshape(-1, 300, 12, 1)
"""

"\nX_train = np.load('../model_input/train_pitch_x.npz')['arr_0'].reshape(-1, 300, 12, 1)\nX_test = np.load('../model_input/test_pitch_x.npz')['arr_0'].reshape(-1, 300, 12, 1)\nX_valid = np.load('../model_input/valid_pitch_x.npz')['arr_0'].reshape(-1, 300, 12, 1)\n"

In [5]:
X_train = np.concatenate((
    np.load('../model_input/train_pitch_x.npz')['arr_0'].reshape(-1, 300, 12),
    np.load('../model_input/train_timbre_x.npz')['arr_0'].reshape(-1, 300, 12), 
    np.load('../model_input/train_loudness_x.npz')['arr_0'].reshape(-1, 300, 3)), axis=2).reshape(-1, 8100)
X_test = np.concatenate((
    np.load('../model_input/test_pitch_x.npz')['arr_0'].reshape(-1, 300, 12), 
    np.load('../model_input/test_timbre_x.npz')['arr_0'].reshape(-1, 300, 12),
    np.load('../model_input/test_loudness_x.npz')['arr_0'].reshape(-1, 300, 3)), axis=2).reshape(-1, 8100)
X_valid = np.concatenate((
    np.load('../model_input/valid_pitch_x.npz')['arr_0'].reshape(-1, 300, 12),
    np.load('../model_input/valid_timbre_x.npz')['arr_0'].reshape(-1, 300, 12), 
    np.load('../model_input/valid_loudness_x.npz')['arr_0'].reshape(-1, 300, 3)), axis=2).reshape(-1, 8100)

In [6]:
#X_train = np.load('../model_input/train_pitch_x.npz')['arr_0'].reshape(-1, 300, 12)
#X_test = np.load('../model_input/test_pitch_x.npz')['arr_0'].reshape(-1, 300, 12)
#X_valid = np.load('../model_input/valid_pitch_x.npz')['arr_0'].reshape(-1, 300, 12)

In [7]:
X_train.shape

(11107, 8100)

In [8]:
#y_test = y_test[:, :1]
#y_train = y_train[:, :1]
#y_valid = y_valid[:, :1]

In [9]:
y_test

array([0, 1, 1, ..., 1, 1, 0], dtype=int64)

In [10]:
X_all = np.concatenate((X_train, X_test, X_valid), axis=0)
dense_enc = MinMaxScaler(feature_range=(0, 1))
dense_enc.fit(X_all)
X_train = dense_enc.transform(X_train).reshape(-1, 300, 27, 1)
X_test = dense_enc.transform(X_test).reshape(-1, 300, 27, 1)
X_valid = dense_enc.transform(X_valid).reshape(-1, 300, 27, 1)

In [12]:
y_all = np.concatenate((y_train, y_test, y_valid), axis=0)
neg, pos = np.bincount(np.squeeze(y_all))
total = neg + pos
weight_for_0 = (1 / neg) * (total) / 2.0
weight_for_1 = (1 / pos) * (total) / 2.0

class_weight = {0: weight_for_0, 1: weight_for_1}
print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

Weight for class 0: 1.19
Weight for class 1: 0.86


In [13]:
#y_all = np.concatenate((y_train, y_test, y_valid), axis=0)
#label_enc = MinMaxScaler(feature_range=(0, 1))
#dense_enc.fit(y_all)
#y_train = dense_enc.transform(y_train)
#y_test = dense_enc.transform(y_test)
#y_valid = dense_enc.transform(y_valid)

In [14]:
EPOCHS = 200
BATCH_SIZE = 128

In [15]:
from keras import backend as K

def r2(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred), axis = 0) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true, axis = 0)), axis = 0) 
    return K.mean(1 - SS_res/(SS_tot + K.epsilon()), axis = -1 )


In [16]:
        """
        TimeDistributed(
            Conv1D(filters=64, kernel_size=3, padding='same', activation='relu', activity_regularizer=regularizers.l2(1e-3),
        input_shape=(27, 1))),
        TimeDistributed(
            MaxPooling1D(pool_size=3),\
        ),
        TimeDistributed(
            Dropout(0.5),
        ),
        TimeDistributed(
            Flatten(),
        ),
        """

"\nTimeDistributed(\n    Conv1D(filters=64, kernel_size=3, padding='same', activation='relu', activity_regularizer=regularizers.l2(1e-3),\ninput_shape=(27, 1))),\nTimeDistributed(\n    MaxPooling1D(pool_size=3),),\nTimeDistributed(\n    Dropout(0.5),\n),\nTimeDistributed(\n    Flatten(),\n),\n"

In [17]:
def build_model():
    model = keras.Sequential([
        TimeDistributed(
            Conv1D(filters=32, kernel_size=3, padding='same', activation='relu', activity_regularizer=regularizers.l2(1e-3),
        input_shape=(27, 1))),
        TimeDistributed(
            MaxPooling1D(pool_size=3),\
        ),
        TimeDistributed(
            Dropout(0.5),
        ),
        TimeDistributed(
            Flatten(),
        ),
        LSTM(32, input_shape=(300, 27), dropout=0.5, recurrent_dropout=0.5),
        Dense(1, activation='sigmoid')
        ])
    adam = optimizers.Adam(lr=1e-3)
    #model.compile(loss='mean_squared_error', optimizer=adam, metrics=[r2])
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

In [18]:
model = build_model()

In [19]:
#es = keras.callbacks.EarlyStopping(monitor='val_r2', mode='max', verbose=1, patience=5, restore_best_weights=True)
es = keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=5, restore_best_weights=True)
#ckpt = keras.callbacks.ModelCheckpoint(filepath='weights.hdf5', verbose=1, save_best_only=True, monitor='val_r2', mode='max')

In [20]:
history = model.fit(
  X_train, y_train, batch_size=BATCH_SIZE,
    epochs=EPOCHS, shuffle=False, class_weight=class_weight,
    validation_data=(X_valid, y_valid),
    verbose=1,
    callbacks=[
        #ckpt, 
        es])

Train on 11107 samples, validate on 3811 samples
Epoch 1/200
11107/11107 [==============================] - 35s 3ms/step - loss: 116.1922 - accuracy: 0.5006 - val_loss: 22.8567 - val_accuracy: 0.5707
Epoch 2/200
11107/11107 [==============================] - 32s 3ms/step - loss: 9.8769 - accuracy: 0.5119 - val_loss: 3.9917 - val_accuracy: 0.5715
Epoch 3/200
11107/11107 [==============================] - 34s 3ms/step - loss: 2.6990 - accuracy: 0.5403 - val_loss: 1.8450 - val_accuracy: 0.5712
Epoch 4/200
11107/11107 [==============================] - 35s 3ms/step - loss: 1.5007 - accuracy: 0.5385 - val_loss: 1.2259 - val_accuracy: 0.5712
Epoch 5/200
11107/11107 [==============================] - 34s 3ms/step - loss: 1.0909 - accuracy: 0.5549 - val_loss: 0.9725 - val_accuracy: 0.5712
Epoch 6/200
11107/11107 [==============================] - 33s 3ms/step - loss: 0.9106 - accuracy: 0.5488 - val_loss: 0.8519 - val_accuracy: 0.5712
Epoch 7/200
11107/11107 [==============================] - 3

In [21]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 300, 27, 32)       128       
_________________________________________________________________
time_distributed_2 (TimeDist (None, 300, 9, 32)        0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 300, 9, 32)        0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 300, 288)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                41088     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 41,249
Trainable params: 41,249
Non-trainable params: 0
__________________________________________________

In [22]:
y_pred = model.predict(X_test, batch_size=BATCH_SIZE)

In [24]:
accuracy_score(y_test, y_pred.round())

0.5782332563510393

In [ ]:
r2_score(y_test, y_pred)

In [ ]:
y_pred_val = model.predict(X_valid, batch_size=BATCH_SIZE)

In [ ]:
r2_score(y_valid, y_pred_val)

In [ ]:
r2_valence = r2_score(y_test[:, :1], y_pred[:, :1])

In [ ]:
r2_valence